### Multiclass Classification

In [1]:
from tables import *
import pandas as pd
import numpy as np
import tensorflow as tf

c:\python\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load data

In [2]:
train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

#View training data
#train.shape (45324, 101)
test.index

Int64Index([45324, 45325, 45326, 45327, 45328, 45329, 45330, 45331, 45332,
            45333,
            ...
            53451, 53452, 53453, 53454, 53455, 53456, 53457, 53458, 53459,
            53460],
           dtype='int64', length=8137)

In [3]:
y_train = train['y']
x_train = train._drop_axis(['y'], axis=1)

#Switch to numpy
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(test)

In [4]:
num_features = len(x_train[1])
total = 0

#standardize data, so for each x1..x100, do (x-mean)/sd(x) s.t. mean(x) ~= 0
for i in range(num_features):
    x_train[:,i] = (x_train[:,i] - np.mean(x_train[:,i])) / np.std(x_train[:,i])
    x_test[:,i] = (x_test[:,i] - np.mean(x_test[:,i])) / np.std(x_test[:,i])
    total += np.mean(x_train[:,i])


#adding all the means, we still have a very small # close to 0
total

-0.00011446602163711628

## KERAS ATTEMPT

In [5]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l2

#Define model
model = Sequential([Dense(70, input_shape=(100,)), #a bit counterintuitve
                   Activation('relu'),
                   Dense(30),
                   Activation('relu'),
                   Dense(50),
                   Activation('relu'),
                   Dense(20),
                   Activation('relu'),
                   Dense(5),
                   Activation('softmax'),
])

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [6]:
### Training
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Easy way to convert to one-hot
y_train_hot2 = keras.utils.to_categorical(y_train, num_classes=5)

model.fit(x_train, y_train_hot2, epochs=75)

Epoch 1/75
45324/45324 [==============================] - 1s 33us/step - loss: 0.4981 - acc: 0.8135
Epoch 2/75
45324/45324 [==============================] - 1s 32us/step - loss: 0.3223 - acc: 0.8850
Epoch 3/75
45324/45324 [==============================] - 1s 33us/step - loss: 0.2851 - acc: 0.8996
Epoch 4/75
45324/45324 [==============================] - 2s 34us/step - loss: 0.2633 - acc: 0.9070
Epoch 5/75
45324/45324 [==============================] - 2s 34us/step - loss: 0.2453 - acc: 0.9138
Epoch 6/75
45324/45324 [==============================] - 1s 33us/step - loss: 0.2355 - acc: 0.9170
Epoch 7/75
45324/45324 [==============================] - 1s 33us/step - loss: 0.2252 - acc: 0.9204
Epoch 8/75
45324/45324 [==============================] - 1s 33us/step - loss: 0.2170 - acc: 0.9228
Epoch 9/75
45324/45324 [==============================] - 2s 33us/step - loss: 0.2075 - acc: 0.9272
Epoch 10/75
45324/45324 [==============================] - 1s 33us/step - loss: 0.2045 - acc: 0.9276

In [7]:
pred_one_hot = model.predict(x_test)
type(pred_one_hot)

numpy.ndarray

In [8]:
#Convert predictions back from 1-hot
pred = []

for line in pred_one_hot:
    pred.append(np.argmax(line))

In [9]:
d = {'Id': test.index, 'y': pred}
out = pd.DataFrame(d)
out.to_csv('output.csv', index=False)

In [10]:
#d = pd.DataFrame([1,3])

In [11]:
#d.to_csv('output_' + '.5' + '.csv', index=False)

In [12]:
#foo = 5

In [13]:
#d.to_csv('output_' + foo + '.csv')